In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
random.seed(42)
np.random.seed(42)

Here, we have loaded the data and set Furan as the label.
At first, we have used 25 percent of the dataset A as the test set to come up with a good model, and then use this model to test in the dataset B.

In [2]:
ds_A = pd.read_csv("/kaggle/input/transformer/DatasetA.csv")
ds_B = pd.read_csv("/kaggle/input/transformer/DatasetB.csv")

ds_A_new = ds_A.drop(set(ds_A.columns) - set(ds_B.columns), axis=1)
ds_B_new = ds_B[ds_A_new.columns]

ds_mix = pd.concat([ds_A_new, ds_B_new]).sample(frac=1).reset_index(drop=True)
y_mix = ds_mix['Furan']
X_mix = ds_mix.drop(["Furan", "HI"], axis = 1)

# For Dataset B
y_B = ds_B_new['Furan']
X_B = ds_B_new.drop(["Furan", "HI"], axis = 1)

# The code below is for the second case, where we train the data for the whole
# Dataset A and test it on Dataset B
y_A = ds_A_new['Furan']
X_A = ds_A_new.drop(["Furan", "HI"], axis = 1)

y_stratify = pd.DataFrame(y_mix.copy())
bins = [-1, 0.1, 1, 100]
labels = [0, 1, 2]
y_stratify['Class'] = pd.cut(y_stratify['Furan'], bins=bins, labels=labels)
y_stratify = y_stratify.drop(["Furan"], axis = 1) 

In [3]:
#ds_mix.hist(bins=50, figsize=(20,15))

The code below performs feature selection via step-wise regression or SequentialFeatureSelector. "n_features_to_select" is a hyperparameter that defines how many feature we'd like to keep which needs to be fine tuned. In our experiment, 5 or 7 features to keep, results in better models. Here we used 7.

In [4]:
'''from sklearn.feature_selection import SequentialFeatureSelector
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

for i in range(1,9):
    xgb_reg = XGBRegressor(random_state=42)

    sfs = SequentialFeatureSelector(xgb_reg,
                                    n_features_to_select=i,
                                    direction='forward',
                                    scoring='r2',
                                    cv=5)

    sfs.fit(X_mix, y_mix)

    # Print the selected features
    print("Selected features" + f"{i}:", sfs.get_feature_names_out())


useful_features1 = ['IFT']
useful_features2 = ['Methane', 'IFT']
useful_features3 = ['Methane', 'Ethane', 'IFT']
useful_features4 = ['Methane', 'Ethane', 'Water', 'IFT']
useful_features5 = ['Methane', 'Ethylene', 'Ethane', 'BDV', 'IFT']
useful_features6 = ['Methane', 'Ethylene', 'Ethane', 'Water', 'BDV', 'IFT']
useful_features7 = ['H2', 'Methane', 'Acetylene', 'Ethylene', 'Ethane', 'BDV', 'IFT']
useful_features8 = ['H2', 'Methane', 'Ethylene', 'Ethane', 'Water', 'Acid', 'BDV', 'IFT']'''

'from sklearn.feature_selection import SequentialFeatureSelector\nfrom catboost import CatBoostRegressor\nfrom xgboost import XGBRegressor\n\nfor i in range(1,9):\n    xgb_reg = XGBRegressor(random_state=42)\n\n    sfs = SequentialFeatureSelector(xgb_reg,\n                                    n_features_to_select=i,\n                                    direction=\'forward\',\n                                    scoring=\'r2\',\n                                    cv=5)\n\n    sfs.fit(X_mix, y_mix)\n\n    # Print the selected features\n    print("Selected features" + f"{i}:", sfs.get_feature_names_out())\n\n\nuseful_features1 = [\'IFT\']\nuseful_features2 = [\'Methane\', \'IFT\']\nuseful_features3 = [\'Methane\', \'Ethane\', \'IFT\']\nuseful_features4 = [\'Methane\', \'Ethane\', \'Water\', \'IFT\']\nuseful_features5 = [\'Methane\', \'Ethylene\', \'Ethane\', \'BDV\', \'IFT\']\nuseful_features6 = [\'Methane\', \'Ethylene\', \'Ethane\', \'Water\', \'BDV\', \'IFT\']\nuseful_features7 = [\'H2

In [5]:
useful_features1 = ['IFT']
useful_features2 = ['Methane', 'IFT']
useful_features3 = ['Methane', 'Ethane', 'IFT']
useful_features4 = ['Methane', 'Ethane', 'Water', 'IFT']
useful_features5 = ['Methane', 'Water', 'Ethane', 'BDV', 'IFT']
useful_features6 = ['Methane', 'Acetylene', 'Ethane', 'Water', 'BDV', 'IFT']
useful_features7 = ['Water', 'Methane', 'Acetylene', 'Ethylene', 'Ethane', 'BDV', 'IFT']
useful_features8 = ['H2', 'Methane', 'Ethylene', 'Ethane', 'Water', 'Acetylene', 'BDV', 'IFT']

In [6]:
Selected_features1= ['Acid']
Selected_features2= ['Acid', 'IFT']
Selected_features3= ['Water', 'Acid', 'IFT']
Selected_features4= ['Water', 'Acid', 'BDV', 'IFT']
Selected_features5= ['Methane', 'Water', 'Acid', 'BDV', 'IFT']
Selected_features6= ['Methane', 'Acetylene', 'Water', 'Acid', 'BDV', 'IFT']
Selected_features7= ['Methane', 'Acetylene', 'Ethane', 'Water', 'Acid', 'BDV', 'IFT']
Selected_features8= ['H2', 'Methane', 'Acetylene', 'Ethane', 'Water', 'Acid', 'BDV', 'IFT']

# First case: Using stratified cross-validation (Round-Robin)

We have experimented a combination of different models in the ensemble.
Although the results were quite similar, we found that a combination of KNN, svm, mlp and logistic regression works best.
In the code below we have created a voting classifier consist of these models.

**Using XGB and without any feature selection**

In [7]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

# define the number of folds
n_splits = 5

# create an instance of StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=n_splits, random_state=42)

# create an empty list to store the mean squared error for each fold
mse_list = []

# iterate over the splits and train/test the model on each split
for train_index, test_index in sss.split(X_mix, y_stratify):
    
    # get the training and testing data for this split
    X_train, X_test = X_mix.loc[train_index], X_mix.loc[test_index]
    y_train, y_test = y_mix.loc[train_index], y_mix.loc[test_index]
    
    # create and fit the model
    xgb_reg = XGBRegressor(random_state=42)
    xgb_reg.fit(X_train, y_train)
    
    # make predictions on the test data
    y_pred = xgb_reg.predict(X_test)
    
    # calculate the mean squared error and add it to the list
    print(mean_squared_error(y_test, y_pred))
    mse_list.append(mean_squared_error(y_test, y_pred))

# calculate the mean of the mean squared error across all the folds
mse_mean = np.mean(mse_list)
print('Mean absolute error A:', np.mean(np.abs(y_pred - y_test)))
print('Mean squared error A:',mse_mean)

0.3000141765494915
0.7651507953826207
0.36220412933485957
0.5272956632763538
0.37799596779911054
Mean absolute error A: 0.26045975661355963
Mean squared error A: 0.46653214646848723


**Using XGB and feature selection**

In [8]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

# define the number of folds
n_splits = 5

# create an instance of StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=n_splits, random_state=42)

# create an empty list to store the mean squared error for each fold
mse_list = []

# iterate over the splits and train/test the model on each split
for train_index, test_index in sss.split(X_mix, y_stratify):
    
    # get the training and testing data for this split
    X_train, X_test = X_mix.loc[train_index], X_mix.loc[test_index]
    y_train, y_test = y_mix.loc[train_index], y_mix.loc[test_index]
    
    # create and fit the model
    xgb_reg = XGBRegressor(random_state=42)
    xgb_reg.fit(X_train[Selected_features8], y_train)
    
    # make predictions on the test data
    y_pred = xgb_reg.predict(X_test[Selected_features8])
    
    # calculate the mean squared error and add it to the list
    print(mean_squared_error(y_test, y_pred))
    mse_list.append(mean_squared_error(y_test, y_pred))

# calculate the mean of the mean squared error across all the folds
mse_mean = np.mean(mse_list)
print('Mean absolute error A:', np.mean(np.abs(y_pred - y_test)))
print('Mean squared error A:',mse_mean)

0.27882777599039704
0.6424226094395493
0.3333520431218333
0.6679665143431887
0.3162367878347854
Mean absolute error A: 0.24968449373453533
Mean squared error A: 0.44776114614595075


**Using CatBoost**

In [9]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor

# define the number of folds
n_splits = 5

# create an instance of StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=n_splits, random_state=42)

# create an empty list to store the mean squared error for each fold
mse_list = []

# iterate over the splits and train/test the model on each split
for train_index, test_index in sss.split(X_mix, y_stratify):
    
    # get the training and testing data for this split
    X_train, X_test = X_mix.loc[train_index], X_mix.loc[test_index]
    y_train, y_test = y_mix.loc[train_index], y_mix.loc[test_index]
    
    # create and fit the model
    cat_reg = CatBoostRegressor(verbose = 0, loss_function='RMSE', random_seed=42)
    cat_reg.fit(X_train, y_train)
    
    # make predictions on the test data
    y_pred = cat_reg.predict(X_test)
    
    # calculate the mean squared error and add it to the list
    print(mean_squared_error(y_test, y_pred))
    mse_list.append(mean_squared_error(y_test, y_pred))

# calculate the mean of the mean squared error across all the folds
mse_mean = np.mean(mse_list)
print('Mean absolute error A:', np.mean(np.abs(y_pred - y_test)))
print('Mean squared error A:',mse_mean)

0.34209066867139426
1.0190729791694189
0.2188961034811211
0.6595825127033147
0.32689179127579165
Mean absolute error A: 0.24550323073216282
Mean squared error A: 0.5133068110602081


**Using catboost and feature selection**

In [10]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor

# define the number of folds
n_splits = 5

# create an instance of StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=n_splits, random_state=42)

# create an empty list to store the mean squared error for each fold
mse_list = []

# iterate over the splits and train/test the model on each split
for train_index, test_index in sss.split(X_mix, y_stratify):
    
    # get the training and testing data for this split
    X_train, X_test = X_mix.loc[train_index], X_mix.loc[test_index]
    y_train, y_test = y_mix.loc[train_index], y_mix.loc[test_index]
    
    # create and fit the model
    cat_reg = CatBoostRegressor(verbose = 0, loss_function='RMSE', random_seed=42)
    cat_reg.fit(X_train[Selected_features8], y_train)
    
    # make predictions on the test data
    y_pred = cat_reg.predict(X_test[Selected_features8])
    
    # calculate the mean squared error and add it to the list
    print(mean_squared_error(y_test, y_pred))
    mse_list.append(mean_squared_error(y_test, y_pred))

# calculate the mean of the mean squared error across all the folds
mse_mean = np.mean(mse_list)
print('Mean absolute error A:', np.mean(np.abs(y_pred - y_test)))
print('Mean squared error A:',mse_mean)

0.29001227707922894
1.0293897540790073
0.24982988739828568
0.6346878078987841
0.2736885227255909
Mean absolute error A: 0.22448906356806927
Mean squared error A: 0.4955216498361793


# Using 75% for train and 25% for test (Stratified test set)

In [11]:
from sklearn.model_selection import train_test_split
train_mix, test_mix = train_test_split(ds_mix, test_size = 0.25, random_state = 42, stratify=y_stratify)

y_train = train_mix['Furan']
X_train = train_mix.drop(['Furan', 'HI'], axis=1)

y_test = test_mix['Furan']
X_test = test_mix.drop(['Furan', 'HI'], axis=1)

**With Catboost**

In [12]:
cat_reg = CatBoostRegressor(verbose = 0, loss_function='RMSE', random_seed=42)
cat_reg.fit(X_train, y_train)

# make predictions on the test data
y_pred = cat_reg.predict(X_test)

# calculate the mean squared error and add it to the list
print('Mean absolute error A:', np.mean(np.abs(y_pred - y_test)))
print(mean_squared_error(y_test, y_pred))

Mean absolute error A: 0.30543095425565536
0.5187898978005979


**With Random Forest**

In [13]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42, n_estimators=800, min_samples_leaf=2)
rf.fit(X_train, y_train)

# make predictions on the test data
y_pred = rf.predict(X_test)

# calculate the mean squared error and add it to the list
print('Mean absolute error A:', np.mean(np.abs(y_pred - y_test)))
print('Mean squared error A:', mean_squared_error(y_test, y_pred))

Mean absolute error A: 0.2956884487284601
Mean squared error A: 0.5512607577165163


**With XGBoost**

In [14]:
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=42, learning_rate= 0.15, gamma=0.5, reg_alpha=1.0)
xgb.fit(X_train, y_train)

# make predictions on the test data
y_pred = xgb.predict(X_test)

# calculate the mean squared error and add it to the list
print('Mean absolute error A:', np.mean(np.abs(y_pred - y_test)))
print('Mean squared error A:', mean_squared_error(y_test, y_pred))

Mean absolute error A: 0.2734135459532113
Mean squared error A: 0.43990406657519876


# Using 75% for train and 25% for test (no stratify)

In [15]:
from sklearn.model_selection import train_test_split
train_mix, test_mix = train_test_split(ds_mix, test_size = 0.25, random_state = 42)

y_train = train_mix['Furan']
X_train = train_mix.drop(['Furan', 'HI'], axis=1)

y_test = test_mix['Furan']
X_test = test_mix.drop(['Furan', 'HI'], axis=1)

**With CatBoost**

In [16]:
cat_reg = CatBoostRegressor(verbose = 0, loss_function='RMSE', random_seed=42)
cat_reg.fit(X_train, y_train)

# make predictions on the test data
y_pred = cat_reg.predict(X_test)

# calculate the mean squared error and add it to the list
print('Mean absolute error A:', np.mean(np.abs(y_pred - y_test)))
print('Mean squared error A:',mean_squared_error(y_test, y_pred))

Mean absolute error A: 0.34076890427507034
Mean squared error A: 0.7330922387370162


**With Random Forest**

In [17]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42, n_estimators=800, min_samples_leaf=2)
rf.fit(X_train, y_train)

# make predictions on the test data
y_pred = rf.predict(X_test)

# calculate the mean squared error and add it to the list
print('Mean absolute error A:', np.mean(np.abs(y_pred - y_test)))
print('Mean squared error A:', mean_squared_error(y_test, y_pred))

Mean absolute error A: 0.33733986216620415
Mean squared error A: 0.6758888911526967


**With XGBoost**

In [18]:
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=42, learning_rate= 0.15, gamma=0.5, reg_alpha=1.0)
xgb.fit(X_train, y_train)

# make predictions on the test data
y_pred = xgb.predict(X_test)

# calculate the mean squared error and add it to the list
print('Mean absolute error A:', np.mean(np.abs(y_pred - y_test)))
print('Mean squared error A:', mean_squared_error(y_test, y_pred))

Mean absolute error A: 0.32299159541889444
Mean squared error A: 0.5782070387849633


# Blending model

In [19]:
train_valid_mix, test_mix = train_test_split(ds_mix, test_size = 0.25, random_state = 42, stratify=y_stratify)

train_mix, valid_mix = train_test_split(train_valid_mix, test_size = 0.15, random_state = 42, stratify=y_stratify.loc[train_valid_mix.index])

y_train = train_mix['Furan']
X_train = train_mix.drop(['Furan', 'HI'], axis=1)

y_test = test_mix['Furan']
X_test = test_mix.drop(['Furan', 'HI'], axis=1)

y_valid = valid_mix['Furan']
X_valid = valid_mix.drop(['Furan', 'HI'], axis=1)

In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import AdaBoostRegressor

# Train the base models
lr = LinearRegression()
rf = RandomForestRegressor(random_state=42)
xgb = XGBRegressor(random_state=42)
cat = CatBoostRegressor(verbose=0, random_state=42, loss_function='RMSE')
svm = SVR(kernel='linear')
knn = KNeighborsRegressor()
mlp = MLPRegressor(max_iter=5000, random_state=42)
el = ElasticNet(alpha=0.1, l1_ratio=0.1, random_state=42)
ada = AdaBoostRegressor(random_state=42)

lr.fit(X_train[useful_features8], y_train)
rf.fit(X_train[useful_features8], y_train)
xgb.fit(X_train[useful_features8], y_train)
cat.fit(X_train[useful_features8], y_train)
svm.fit(X_train[useful_features8], y_train)
knn.fit(X_train[useful_features8], y_train)
mlp.fit(X_train[useful_features8], y_train)
el.fit(X_train[useful_features8], y_train)
ada.fit(X_train[useful_features8], y_train)

# Make predictions on the test set A
lr_predictions = lr.predict(X_valid[useful_features8])
rf_predictions = rf.predict(X_valid[useful_features8])
xgb_predictions = xgb.predict(X_valid[useful_features8])
cat_predictions = cat.predict(X_valid[useful_features8])
svm_predictions = svm.predict(X_valid[useful_features8])
knn_predictions = knn.predict(X_valid[useful_features8])
el_predictions = el.predict(X_valid[useful_features8])
ada_predictions = ada.predict(X_valid[useful_features8])

# Combine the predictions using a meta model
meta_model = RandomForestRegressor(random_state=42)
meta_model.fit(np.column_stack([lr_predictions,
                                #rf_predictions,
                                xgb_predictions,
                                #cat_predictions,
                                #svm_predictions,
                                knn_predictions,
                                ada_predictions,
                                el_predictions
                                ]), y_valid)

lr_predictions_test = lr.predict(X_test[useful_features8])
rf_predictions_test = rf.predict(X_test[useful_features8])
xgb_predictions_test = xgb.predict(X_test[useful_features8])
cat_predictions_test = cat.predict(X_test[useful_features8])
svm_predictions_test = svm.predict(X_test[useful_features8])
knn_predictions_test = knn.predict(X_test[useful_features8])
el_predictions_test = el.predict(X_test[useful_features8])
ada_predictions_test = ada.predict(X_test[useful_features8])

# Make predictions on the test set using the meta model
blend_predictions = meta_model.predict(np.column_stack([lr_predictions_test,
                                                        #rf_predictions_test,
                                                        xgb_predictions_test,
                                                        #cat_predictions_test,
                                                        #svm_predictions_test,
                                                        knn_predictions_test,
                                                        ada_predictions_test,
                                                        el_predictions_test
                                                       ]))


# Evaluate the model
print('Mean absolute error A:', np.mean(np.abs(blend_predictions - y_test)))
print('Mean squared error A:', np.mean((blend_predictions - y_test)**2))

Mean absolute error A: 0.31663330360377356
Mean squared error A: 0.5280468006895551
